## This note book gives the overall sentiment score towards a gender engaging on emails in a mailing list

What it does:
-it identifies the gender of the person In-Reply-To, and the person who it is from From. It calculates the sentiment of the body of the message. So that is the sentiment score of one gender towards another gender. Normaize over number of messages 

Parameters to set options:

Useful extensions:


In [1]:
%matplotlib inline

In [2]:
from bigbang.archive import load as load_archive
from bigbang.archive import Archive
import bigbang.ingress.mailman as mailman
import bigbang.analysis.process as process
from bigbang import parse
from bigbang import utils

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint as pp
import pytz
import numpy as np
import math
from textblob import TextBlob
from itertools import repeat
from gender_detector import gender_detector as gd
import seaborn as sns
import re


In [3]:
#insert a list of the urls of downloaded mailing lists that you want to include in the analysis. 
#data will be merged: multiple mailing lists are treated as a unique corpus


import os 
cwd = os.getcwd()    

archives_names = ["quic"]


archives_paths = list()
for archive_name in archives_names:
    archives_paths.append('../archives/'+archive_name+'.csv')
    

archives_list = [load_archive(archive_path).data for archive_path in archives_paths]
    
archives = Archive(pd.concat(archives_list))

archives_data = archives.data

In [ ]:
print(archives_data['In-Reply-To'])

In [4]:
detector = gd.GenderDetector('us')

gender_ambiguous_names = {}

def guess_gender(name):
    if not name:
        return 'name unknown'
    try:
        if detector.guess(name) == 'unknown':
            if name in gender_ambiguous_names:
                gender_ambiguous_names[name] += 1
            else:
                gender_ambiguous_names[name] = 1
        
        return detector.guess(name)
    except:
        return 'error'

def ml_shortname(url):
    return url.rstrip("/").split("/")[-1]

In [5]:
archives_data['Body Sentiment'] = archives_data['Body'].map(lambda x: TextBlob(utils.clean_message(x)).sentiment.polarity)
archives_data['InReplyTo Name'] = archives_data['In-Reply-To'].index.map(lambda x: parse.clean_from(x))    
archives_data['From Name'] = archives_data['From'].index.map(lambda x: parse.clean_from(x))    
archives_data['InReplyTo First Name'] = archives_data['InReplyTo Name'].map(lambda x: parse.guess_first_name(parse.clean_name(x)))
archives_data['From First Name'] = archives_data['From Name'].map(lambda x: parse.guess_first_name(parse.clean_name(x)))
archives_data['InReplyTo Gender'] = archives_data['InReplyTo First Name'].map(guess_gender)
archives_data['From Gender'] = archives_data['From First Name'].map(guess_gender)

In [7]:
df = archives_data.groupby(by=['InReplyTo Gender','From Gender'])['Body Sentiment'].aggregate('sum')
print(df)
#sns.heatmap(df, annot=True)

InReplyTo Gender  From Gender 
name unknown      name unknown    521.012970
unknown           unknown         175.265011
Name: Body Sentiment, dtype: float64
